In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 16168084
paper_name = 'eide_harper_2005' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/13059_2005_1112_MOESM2_ESM.xlsx', sheet_name='Normalized concentrations')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 233 x 14


In [7]:
original_data.head()

,Gene Mutated,Ca,Co,Cu,Fe,K,Mg,Mn,Na,Ni,P,S,Se,Zn
0,YPL166W,0.434007,0.108688,0.092560,0.154784,32.1572,2.05665,0.009106,2.231140,1.178060,23.7410,5.24320,0.928303,2.61027
1,YPL193W,0.364949,0.058945,0.093358,0.165285,28.2791,2.15907,0.011589,0.694450,0.398613,24.0892,5.62505,0.942674,1.98651
2,YOL072W,0.386290,0.074596,0.165109,0.178989,26.0944,2.24682,0.011872,0.520811,0.499773,23.9819,5.54915,0.739865,2.24728
3,YGR078C,0.557621,0.059579,0.200603,0.104351,28.0642,2.07379,0.011800,0.993561,0.454281,25.2248,5.34246,0.877306,2.24072
4,YPL179W,0.671418,0.124838,0.111410,0.195118,22.3629,2.61291,0.007517,0.605458,1.381590,24.6645,4.12101,0.724180,2.71106


In [8]:
original_data['orf'] = original_data['Gene Mutated'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
original_data.loc[original_data['orf']=='YILO16W','orf'] = 'YIL016W'

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Gene Mutated,  Ca,  Co,  Cu,  Fe,  K,  Mg,  Mn,  Na,  Ni,  P,  S,  Se,  Zn, orf]
Index: []


In [15]:
original_data.set_index('orf', inplace=True)

In [16]:
original_data.drop(columns='Gene Mutated', inplace=True)

In [17]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(233, 13)

# Load dataset ids

In [22]:
dt = pd.read_excel('extras/datasets.xlsx', sheet_name='Sheet1', header=None)
dt.head()

,0,1
0,Ca,4784
1,Co,4785
2,Cu,4786
3,Fe,4787
4,K,4792


In [23]:
dt.set_index(0, inplace=True)

In [24]:
dt = dt.reindex(index=original_data.columns.values)

In [26]:
dataset_ids = dt[1].values

# Load & process tested strains

In [27]:
tested = pd.read_excel('raw_data/13059_2005_1112_MOESM1_ESM.xlsx', sheet_name='Genes Analyzed')

In [28]:
tested.head()

,First pass (n=1) samples,Second pass (n=3) samples,Third pass (n=6) samples,Genes not analyzed
0,YAL002W,YAL002W,YPL166W,YAL069W
1,YAL004W,YAL011W,YPL193W,YAL068W-A
2,YAL005C,YAL013W,YOL072W,YAL067W-A
3,YAL007C,YAL015C,YGR078C,YAL064W-B
4,YAL008W,YAL017W,YPL179W,YAL064W


In [29]:
tested['orf'] = tested['First pass (n=1) samples'].astype(str)

In [30]:
tested['orf'] = clean_orf(tested['orf'])

In [33]:
typo_fixes = {'YAL022C-2':'YAL022C','YAL022C-3':'YAL022C','YELOO1C':'YEL001C','YIL077C-3':'YIL077C','YOR134W-':'YOR134W'}

In [34]:
tested['orf'] = tested['orf'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)

In [35]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [36]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

            First pass (n=1) samples Second pass (n=3) samples  \
index_input                                                      
3379                          YMR41W                       NaN   

            Third pass (n=6) samples Genes not analyzed     orf  
index_input                                                      
3379                             NaN                NaN  YMR41W  


In [37]:
tested = tested.loc[t,:]

In [38]:
tested_orfs = tested['orf'].unique()

In [39]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

[]

In [40]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [41]:
data = original_data.copy()

In [42]:
datasets = datasets.reindex(index=dataset_ids)

In [43]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [44]:
data.head()

dataset_id,4784,4785,4786,4787,4792,4788,4789,4794,4790,4791,4795,4793,4796
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,
YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL004W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL005C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL007C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YAL008W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [45]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [46]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,4784,4785,4786,4787,4792,4788,4789,4794,4790,4791,4795,4793,4796
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,
2,YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1863,YAL004W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,YAL005C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,YAL007C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,YAL008W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Normalize

In [47]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [48]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [49]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,4784,4785,4786,4787,4792,4788,4789,4794,4790,4791,...,4787,4792,4788,4789,4794,4790,4791,4795,4793,4796
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
2,YAL002W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07958,0.0,0.0,0.0,-0.050218,0.0
1863,YAL004W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07958,0.0,0.0,0.0,-0.050218,0.0
4,YAL005C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07958,0.0,0.0,0.0,-0.050218,0.0
5,YAL007C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07958,0.0,0.0,0.0,-0.050218,0.0
6,YAL008W,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.07958,0.0,0.0,0.0,-0.050218,0.0


# Print out

In [50]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [51]:
from IO.save_data_to_db3 import *

In [52]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/13 [00:00<?, ?it/s]

Deleting all datasets for PMID 16168084...
Inserting the new data...


100%|██████████| 13/13 [01:32<00:00,  7.15s/it]

Updating the data_modified_on field...
